In [14]:
import numpy as np
import math
from stl import mesh
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt

%matplotlib widget

In [15]:
def calculate_3d_distance(vector1, vector2):
    if len(vector1) != 3 or len(vector2) != 3:
        raise ValueError("Both vectors must have exactly 3 coordinates (X, Y, Z).")

    distance = math.sqrt((vector2[0] - vector1[0])**2 +
                         (vector2[1] - vector1[1])**2 +
                         (vector2[2] - vector1[2])**2)

    return distance

def calculate_midpoint(point1, point2):
    if len(point1) != 3 or len(point2) != 3:
        raise ValueError("Both points must have exactly 3 coordinates (X, Y, Z).")

    midpoint = [(point1[0] + point2[0]) / 2,
                (point1[1] + point2[1]) / 2,
                (point1[2] + point2[2]) / 2]

    return midpoint

def cross_product(v1, v2):
    result = [v1[1] * v2[2] - v1[2] * v2[1],
              v1[2] * v2[0] - v1[0] * v2[2],
              v1[0] * v2[1] - v1[1] * v2[0]]
    
    return np.array(result)

def rotation_matrix(axis, angle):
    # Make sure the axis is a unit vector
    axis = np.array(axis)
    axis /= np.linalg.norm(axis)

    # Calculate the rotation matrix
    x, y, z = axis
    c = math.cos(angle)
    s = math.sin(angle)
    t = 1 - c

    matrix = np.array([[t * x**2 + c,      t * x * y - s * z, t * x * z + s * y],
                       [t * x * y + s * z, t * y**2 + c,      t * y * z - s * x],
                       [t * x * z - s * y, t * y * z + s * x, t * z**2 + c]])

    return matrix

def rad2deg(x):
    return x / math.pi * 180

def dot(vector1, vector2):
    if len(vector1) != len(vector2):
        raise ValueError("Vectors must have the same length.")

    dot_product = sum(x * y for x, y in zip(vector1, vector2))
    return dot_product

In [16]:
S = mesh.Mesh.from_file('Mouse.stl')

In [17]:
Left = np.array([3.418143, -0.719809, -0.39438])
Right = np.array([3.42447, -6.569044, 0.957253])
Ant = np.array([-11.445497, -1.679053, 7.836357])

In [18]:
Mid = calculate_midpoint(Left, Right)
Ant_zero = Ant - Mid
print(Mid)
print(Ant_zero)

[3.4213065, -3.6444265, 0.2814365]
[-14.8668035   1.9653735   7.5549205]


In [19]:
# Translate to the midpoint between L and R
S.x += -1 * Mid[0]
S.y += -1 * Mid[1]
S.z += -1 * Mid[2]

In [20]:
# Rotation matrix
v1 = [1, 0, 0]
axis_of_rotation = cross_product(v1, Ant_zero)
unit_axis = axis_of_rotation / np.linalg.norm(axis_of_rotation)
unit_axis

array([ 0.        , -0.96778836,  0.25176514])

In [21]:
angle_of_rotation = math.acos(dot(v1, Ant_zero) / (np.linalg.norm(v1) * np.linalg.norm(Ant_zero)))
angle_of_rotation

2.6580768502943943

In [22]:
R = rotation_matrix(unit_axis, angle_of_rotation)
R

array([[-0.88536592, -0.11704431, -0.44991979],
       [ 0.11704431,  0.88049479, -0.45937953],
       [ 0.44991979, -0.45937953, -0.76586071]])

In [23]:
# S.rotate(axis = np.array([ 0, 0.9715672, -0.2367641 ]), theta = 4.0507656, point = np.array([0, 0, 0]))
# R = np.array([ [-0.6143985, -0.1868059, -0.7665625],
#                [ 0.1868059,  0.9095013, -0.3713636],
#                [ 0.7665625, -0.3713636, -0.5238998]])
# R

In [24]:
S.rotate_using_matrix(R)

In [25]:
S.save('Mouse_rot.stl')